# Spark SQL Quiz

This quiz uses the same dataset and questions from the Spark Data Frames Programming Quiz. For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, IntegerType
from pyspark.sql.functions import desc, asc, udf, avg, count, sort_array, when, col, isnan
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
spark = SparkSession \
    .builder \
    .appName('Data Wrangling') \
    .getOrCreate()

In [3]:
path = "data/sparkify_log_small.json"
log = spark.read.json(path)
log.createOrReplaceTempView('log')

# Question 1

Which page did user id ""(empty string) NOT visit?

In [14]:
query = """
        SELECT * 
        FROM 
        (SELECT DISTINCT page
        FROM log 
        WHERE userId = '') AS blank

        RIGHT JOIN
        (SELECT DISTINCT page
        FROM log) AS all
        ON blank.page = all.page
        WHERE blank.page IS NULL
        """

In [16]:
spark.sql(query).show()

+----+----------------+
|page|            page|
+----+----------------+
|null|Submit Downgrade|
|null|       Downgrade|
|null|          Logout|
|null|   Save Settings|
|null|        Settings|
|null|        NextSong|
|null|         Upgrade|
|null|           Error|
|null|  Submit Upgrade|
+----+----------------+



# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

# Question 3

How many female users do we have in the data set?

In [10]:
query = """
        SELECT count(DISTINCT userId)
        FROM log 
        WHERE gender = 'F'
        """
spark.sql(query).show()

+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                   462|
+----------------------+



# Question 4

How many songs were played from the most played artist?

In [12]:
query = """
        SELECT artist, count(song)
        FROM log 
        GROUP BY artist
        ORDER BY count(song) DESC
        LIMIT 10
        """
spark.sql(query).show()

+--------------------+-----------+
|              artist|count(song)|
+--------------------+-----------+
|            Coldplay|         83|
|       Kings Of Leon|         69|
|Florence + The Ma...|         52|
|            BjÃÂ¶rk|         46|
|       Dwight Yoakam|         45|
|       Justin Bieber|         43|
|      The Black Keys|         40|
|         OneRepublic|         37|
|        Jack Johnson|         36|
|                Muse|         36|
+--------------------+-----------+



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [25]:
query1 = """
        SELECT userId, ts, page, 
        CASE WHEN page = 'Home' THEN 1 else 0 END AS home_page
        FROM log
        WHERE (page = 'Home') or (page ='NextSong') 
        """
table1 = spark.sql(query1)
table1.createOrReplaceTempView('table1')
table1.show()

+------+-------------+--------+---------+
|userId|           ts|    page|home_page|
+------+-------------+--------+---------+
|  1046|1513720872284|NextSong|        0|
|  1000|1513720878284|NextSong|        0|
|  2219|1513720881284|NextSong|        0|
|  2373|1513720905284|NextSong|        0|
|  1747|1513720913284|    Home|        1|
|  1162|1513720955284|NextSong|        0|
|  1061|1513720959284|NextSong|        0|
|   748|1513720959284|    Home|        1|
|   597|1513720980284|    Home|        1|
|  1806|1513720983284|NextSong|        0|
|   748|1513720993284|NextSong|        0|
|  1176|1513721031284|NextSong|        0|
|  2164|1513721045284|NextSong|        0|
|  2146|1513721058284|NextSong|        0|
|  2219|1513721077284|NextSong|        0|
|  1176|1513721088284|    Home|        1|
|  2904|1513721095284|NextSong|        0|
|   597|1513721097284|NextSong|        0|
|   226|1513721104284|NextSong|        0|
|  1046|1513721104284|NextSong|        0|
+------+-------------+--------+---

In [26]:
query2 ="""
        SELECT *, SUM(home_page) OVER(PARTITION BY userId ORDER BY ts ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS period
        FROM table1  
        """
table2 = spark.sql(query2)
table2.createOrReplaceTempView('table2')
table2.show(100)

+------+-------------+--------+---------+------+
|userId|           ts|    page|home_page|period|
+------+-------------+--------+---------+------+
|  1436|1513782858284|NextSong|        0|     0|
|  1436|1513783259284|NextSong|        0|     0|
|  2088|1513802824284|NextSong|        0|     0|
|  2088|1513803057284|NextSong|        0|     0|
|  2088|1513803254284|NextSong|        0|     0|
|  2088|1513803413284|NextSong|        0|     0|
|  2088|1513803651284|NextSong|        0|     0|
|  2088|1513803820284|NextSong|        0|     0|
|  2088|1513803967284|NextSong|        0|     0|
|  2088|1513804196284|NextSong|        0|     0|
|  2088|1513804555284|NextSong|        0|     0|
|  2088|1513804786284|NextSong|        0|     0|
|  2088|1513805065284|NextSong|        0|     0|
|  2088|1513805494284|NextSong|        0|     0|
|  2088|1513805859284|NextSong|        0|     0|
|  2088|1513805972284|    Home|        1|     1|
|  2162|1513773170284|NextSong|        0|     0|
|  2162|151377327128

In [29]:
query3 = """
        SELECT AVG(num_song)
        FROM
        (SELECT count(*) AS num_song
        FROM table2
        GROUP BY userId, period, page
        HAVING page = 'NextSong') AS table3
        """
spark.sql(query3).show()

+------------------+
|     avg(num_song)|
+------------------+
|6.9558333333333335|
+------------------+

